In [1]:
import pandas as pd
import seaborn as sbs
import numpy as np
import calendar
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score

In [2]:
properati = pd.read_csv("data/properati_full_month.csv",low_memory=False)

In [3]:
test = pd.read_csv("data/properati_dataset_testing_noprice.csv")

In [4]:
test.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


In [5]:
test.shape

(14166, 17)

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [7]:
properati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640710 entries, 0 to 640709
Data columns (total 29 columns):
country_name                  263989 non-null object
created_on                    640710 non-null object
currency                      532703 non-null object
description                   428806 non-null object
expenses                      37716 non-null object
extra                         164828 non-null object
floor                         80294 non-null float64
geonames_id                   541207 non-null float64
id                            263989 non-null object
image_thumbnail               620269 non-null object
lat                           480309 non-null float64
lat-lon                       480309 non-null object
lon                           480304 non-null float64
operation                     640710 non-null object
place_name                    640658 non-null object
place_with_parent_names       640710 non-null object
price                         540315 no

# Dropeo de columnas, asi quedan misma cantidad de columnas en el train ( properati ) y en el test, solo me quedo con price_aprox_usd

In [8]:
properati.drop(inplace=True,labels=\
               ['image_thumbnail','title','surface_in_m2','price_aprox_local_currency','price',\
               'price_per_m2','price_usd_per_m2','properati_url','title','geonames_id','currency','extra'],axis=1)

In [9]:
properati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640710 entries, 0 to 640709
Data columns (total 18 columns):
country_name               263989 non-null object
created_on                 640710 non-null object
description                428806 non-null object
expenses                   37716 non-null object
floor                      80294 non-null float64
id                         263989 non-null object
lat                        480309 non-null float64
lat-lon                    480309 non-null object
lon                        480304 non-null float64
operation                  640710 non-null object
place_name                 640658 non-null object
place_with_parent_names    640710 non-null object
price_aprox_usd            540313 non-null float64
property_type              640710 non-null object
rooms                      363676 non-null float64
state_name                 263989 non-null object
surface_covered_in_m2      226561 non-null float64
surface_total_in_m2        185630 n

In [10]:
test['country_name'].value_counts(dropna=False)

Argentina    14166
Name: country_name, dtype: int64

In [11]:
test['lat'].value_counts(dropna=False)

NaN           3679
-34.611058      78
-34.619792      68
-34.589546      66
-34.612229      60
-34.709935      57
-34.571150      51
-34.662022      50
-34.629799      44
-34.622973      44
-34.595587      43
-34.624516      38
-34.551305      36
-34.587405      34
-34.573524      30
-34.609438      30
-34.657024      27
-34.610122      27
-34.597773      26
-34.562084      25
-34.613968      23
-34.605859      22
-34.643085      21
-34.630663      20
-34.637213      19
-34.648101      19
-34.613306      18
-34.615189      18
-34.618621      17
-34.560664      17
              ... 
-34.802527       1
-34.646519       1
-34.611639       1
-34.570697       1
-34.546028       1
-34.635246       1
-34.645187       1
-34.597221       1
-34.664200       1
-34.578133       1
-34.595148       1
-34.647684       1
-34.587427       1
-34.555612       1
-34.579530       1
-34.737039       1
-34.596994       1
-34.815759       1
-34.585870       1
-34.635494       1
-34.623779       1
-34.635656  

In [12]:
test['state_name'].value_counts(dropna=False)

Capital Federal             8847
Bs.As. G.B.A. Zona Oeste    3101
Bs.As. G.B.A. Zona Sur      2215
Bs.As. G.B.A. Zona Norte       2
Buenos Aires Interior          1
Name: state_name, dtype: int64

In [13]:
test['place_name'].value_counts(dropna=False)

Palermo                       1020
Belgrano                       783
Caballito                      650
La Plata                       603
Recoleta                       556
Ituzaingó                      547
Ramos Mejía                    394
Morón                          393
Almagro                        376
Castelar                       345
Flores                         329
Villa Crespo                   329
Balvanera                      327
Villa Urquiza                  309
Nuñez                          300
Barrio Norte                   276
Capital Federal                247
Monserrat                      221
Saavedra                       211
San Telmo                      207
Lomas de Zamora                197
Quilmes                        178
Moreno                         171
Palermo Hollywood              164
Puerto Madero                  155
Banfield                       134
Haedo                          126
Villa Devoto                   123
Lanús               

In [14]:
test['place_with_parent_names'].value_counts()

|Argentina|Capital Federal|Palermo|                                          1020
|Argentina|Capital Federal|Belgrano|                                          783
|Argentina|Capital Federal|Caballito|                                         650
|Argentina|Capital Federal|Recoleta|                                          556
|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|La Plata|                          493
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|                      490
|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza|Ramos Mejía|                   394
|Argentina|Capital Federal|Almagro|                                           376
|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|Castelar|                           345
|Argentina|Capital Federal|Flores|                                            329
|Argentina|Capital Federal|Villa Crespo|                                      329
|Argentina|Capital Federal|Balvanera|                                         327
|Argentina|Capit

In [15]:
test['floor'].value_counts(dropna=False)

NaN        12798
 1.0         242
 2.0         203
 3.0         161
 4.0         150
 5.0         103
 7.0          97
 6.0          93
 8.0          76
 9.0          62
 10.0         35
 11.0         26
 13.0         25
 12.0         22
 14.0         14
 16.0         12
 17.0          7
 15.0          6
 20.0          4
 22.0          3
 26.0          3
 36.0          2
 18.0          2
 303.0         2
 29.0          2
 59.0          1
 603.0         1
 601.0         1
 546.0         1
 231.0         1
 207.0         1
 104.0         1
 21.0          1
 37.0          1
 2103.0        1
 28.0          1
 27.0          1
 24.0          1
 23.0          1
 19.0          1
 2509.0        1
Name: floor, dtype: int64

# Parseo de datos

In [16]:
properati.country_name = properati.place_with_parent_names.apply(lambda x : x.split('|')[1])

In [17]:
properati = properati[properati['country_name'] == 'Argentina']

In [18]:
states = properati.place_with_parent_names.apply(lambda x : x.split('|')[2])

In [19]:
properati.loc[:,'state_name'] = states

In [20]:
properati['state_name'].value_counts()

Capital Federal                 196351
Bs.As. G.B.A. Zona Norte        141007
Buenos Aires Costa Atlántica     68168
Bs.As. G.B.A. Zona Oeste         61735
Bs.As. G.B.A. Zona Sur           47133
Santa Fe                         36506
Córdoba                          35480
Buenos Aires Interior            14128
Río Negro                         3560
Neuquén                           3219
Mendoza                           3023
Entre Ríos                        2973
Misiones                          2793
Tucumán                           1527
Salta                             1525
La Pampa                          1118
San Luis                          1113
Corrientes                         707
Chubut                             507
Chaco                              397
Catamarca                          341
San Juan                           175
Jujuy                              164
Tierra Del Fuego                   154
Santiago Del Estero                 80
Santa Cruz               

In [21]:
properati = properati[
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Norte' ) |
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Sur' ) |
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Oeste' ) |
    (properati['state_name'] == 'Capital Federal' )  |
    (properati['state_name'] == 'Buenos Aires Interior')
    ]

In [22]:
properati['place_name'].value_counts()

Capital Federal                             18865
Belgrano                                    17506
Tigre                                       16740
Nordelta                                    16011
Palermo                                     14459
Pilar                                       14215
Caballito                                   11919
La Plata                                    11851
Morón                                        8312
Olivos                                       8312
Barrio Norte                                 8261
Villa Urquiza                                7974
Villa Crespo                                 7923
Flores                                       7447
Ramos Mejía                                  7189
La Matanza                                   7074
San Isidro                                   7046
Almagro                                      6772
Recoleta                                     6670
Ituzaingó                                    6265


In [23]:
properati['place_name'].count()

460302

In [24]:
properati['zone'] = properati.place_with_parent_names.apply(lambda x : x.split('|')[3])

In [25]:
properati['zone'].value_counts()

Tigre                   45521
                        24544
Palermo                 22482
Vicente López           21566
La Matanza              20195
Pilar                   18257
San Isidro              17795
Belgrano                17506
Morón                   16671
La Plata                14707
Caballito               11919
General San Martín       9673
Tres de Febrero          8901
Barrio Norte             8261
Villa Urquiza            7974
Villa Crespo             7923
Escobar                  7654
Ituzaingó                7622
Lanús                    7540
Flores                   7447
Lomas de Zamora          7145
San Fernando             7077
Almagro                  6772
Recoleta                 6670
Nuñez                    6120
San Miguel               5723
Boedo                    4697
Balvanera                4397
Quilmes                  4344
Almirante Brown          3814
                        ...  
General Lamadrid            9
Mar Chiquita                8
Ramallo   

In [26]:
properati['zone'].count()

460354

In [27]:
properati.drop(inplace=True,labels=['zone'],axis=1)

In [28]:
del properati['country_name']
del properati['description']
del properati['expenses']
del properati['id']
del properati['place_with_parent_names']
del properati['operation']

# Prueba de KNN

In [29]:
train = properati

In [30]:
train

,created_on,floor,lat,lat-lon,lon,place_name,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2
0,2013-08-01,1.0,-34.598942,"-34.598942,-58.500647",-58.500647,Capital Federal,130896.0,apartment,2.0,Capital Federal,NaN,NaN
1,2013-08-01,NaN,NaN,NaN,NaN,Campana,205000.0,house,2.0,Buenos Aires Interior,NaN,NaN
2,2013-08-01,NaN,-34.591586,"-34.591586,-58.385966",-58.385966,Recoleta,595000.0,apartment,2.0,Capital Federal,NaN,NaN
3,2013-08-01,NaN,-34.587855,"-34.587855,-58.395342",-58.395342,Recoleta,140000.0,apartment,2.0,Capital Federal,NaN,NaN
4,2013-08-01,NaN,-34.597944,"-34.597944,-58.415853",-58.415853,Palermo,118000.0,apartment,2.0,Capital Federal,NaN,NaN
5,2013-08-02,NaN,NaN,NaN,NaN,Berazategui,800000.0,house,5.0,Bs.As. G.B.A. Zona Sur,NaN,NaN
6,2013-08-02,2.0,-34.618726,"-34.618726,-58.403444",-58.403444,San Cristobal,70300.0,apartment,1.0,Capital Federal,NaN,NaN
7,2013-08-02,NaN,-34.590938,"-34.590938,-58.391122",-58.391122,Recoleta,600000.0,apartment,3.0,Capital Federal,NaN,NaN
8,2013-08-02,NaN,NaN,NaN,NaN,Pilar,1100000.0,house,5.0,Bs.As. G.B.A. Zona Norte,NaN,NaN
10,2013-08-02,NaN,-34.592961,"-34.592961,-58.376981",-58.376981,Barrio Norte,445000.0,apartment,2.0,Capital Federal,NaN,NaN


In [31]:
properati

,created_on,floor,lat,lat-lon,lon,place_name,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2
0,2013-08-01,1.0,-34.598942,"-34.598942,-58.500647",-58.500647,Capital Federal,130896.0,apartment,2.0,Capital Federal,NaN,NaN
1,2013-08-01,NaN,NaN,NaN,NaN,Campana,205000.0,house,2.0,Buenos Aires Interior,NaN,NaN
2,2013-08-01,NaN,-34.591586,"-34.591586,-58.385966",-58.385966,Recoleta,595000.0,apartment,2.0,Capital Federal,NaN,NaN
3,2013-08-01,NaN,-34.587855,"-34.587855,-58.395342",-58.395342,Recoleta,140000.0,apartment,2.0,Capital Federal,NaN,NaN
4,2013-08-01,NaN,-34.597944,"-34.597944,-58.415853",-58.415853,Palermo,118000.0,apartment,2.0,Capital Federal,NaN,NaN
5,2013-08-02,NaN,NaN,NaN,NaN,Berazategui,800000.0,house,5.0,Bs.As. G.B.A. Zona Sur,NaN,NaN
6,2013-08-02,2.0,-34.618726,"-34.618726,-58.403444",-58.403444,San Cristobal,70300.0,apartment,1.0,Capital Federal,NaN,NaN
7,2013-08-02,NaN,-34.590938,"-34.590938,-58.391122",-58.391122,Recoleta,600000.0,apartment,3.0,Capital Federal,NaN,NaN
8,2013-08-02,NaN,NaN,NaN,NaN,Pilar,1100000.0,house,5.0,Bs.As. G.B.A. Zona Norte,NaN,NaN
10,2013-08-02,NaN,-34.592961,"-34.592961,-58.376981",-58.376981,Barrio Norte,445000.0,apartment,2.0,Capital Federal,NaN,NaN


In [32]:
train = train[pd.notnull(train['place_name'])]

In [33]:
train = train[pd.notnull(train['price_aprox_usd'])]

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408626 entries, 0 to 640708
Data columns (total 12 columns):
created_on               408626 non-null object
floor                    64388 non-null float64
lat                      304349 non-null float64
lat-lon                  304349 non-null object
lon                      304344 non-null float64
place_name               408626 non-null object
price_aprox_usd          408626 non-null float64
property_type            408626 non-null object
rooms                    225126 non-null float64
state_name               408626 non-null object
surface_covered_in_m2    158659 non-null float64
surface_total_in_m2      127116 non-null float64
dtypes: float64(7), object(5)
memory usage: 40.5+ MB


In [35]:
train

,created_on,floor,lat,lat-lon,lon,place_name,price_aprox_usd,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2
0,2013-08-01,1.0,-34.598942,"-34.598942,-58.500647",-58.500647,Capital Federal,130896.0,apartment,2.0,Capital Federal,NaN,NaN
1,2013-08-01,NaN,NaN,NaN,NaN,Campana,205000.0,house,2.0,Buenos Aires Interior,NaN,NaN
2,2013-08-01,NaN,-34.591586,"-34.591586,-58.385966",-58.385966,Recoleta,595000.0,apartment,2.0,Capital Federal,NaN,NaN
3,2013-08-01,NaN,-34.587855,"-34.587855,-58.395342",-58.395342,Recoleta,140000.0,apartment,2.0,Capital Federal,NaN,NaN
4,2013-08-01,NaN,-34.597944,"-34.597944,-58.415853",-58.415853,Palermo,118000.0,apartment,2.0,Capital Federal,NaN,NaN
5,2013-08-02,NaN,NaN,NaN,NaN,Berazategui,800000.0,house,5.0,Bs.As. G.B.A. Zona Sur,NaN,NaN
6,2013-08-02,2.0,-34.618726,"-34.618726,-58.403444",-58.403444,San Cristobal,70300.0,apartment,1.0,Capital Federal,NaN,NaN
7,2013-08-02,NaN,-34.590938,"-34.590938,-58.391122",-58.391122,Recoleta,600000.0,apartment,3.0,Capital Federal,NaN,NaN
8,2013-08-02,NaN,NaN,NaN,NaN,Pilar,1100000.0,house,5.0,Bs.As. G.B.A. Zona Norte,NaN,NaN
10,2013-08-02,NaN,-34.592961,"-34.592961,-58.376981",-58.376981,Barrio Norte,445000.0,apartment,2.0,Capital Federal,NaN,NaN


In [36]:
train['property_type'].value_counts(dropna=False)

apartment    245624
house        128975
PH            25431
store          8596
Name: property_type, dtype: int64

In [37]:
train['cosa'] = train['property_type'].rank(method='dense', ascending=False).astype(int)

In [38]:
train['cosa'].value_counts()

3    245624
2    128975
4     25431
1      8596
Name: cosa, dtype: int64

In [39]:
train['otro'] = train['place_name'].rank(method='dense', ascending=False).astype(int)

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408626 entries, 0 to 640708
Data columns (total 14 columns):
created_on               408626 non-null object
floor                    64388 non-null float64
lat                      304349 non-null float64
lat-lon                  304349 non-null object
lon                      304344 non-null float64
place_name               408626 non-null object
price_aprox_usd          408626 non-null float64
property_type            408626 non-null object
rooms                    225126 non-null float64
state_name               408626 non-null object
surface_covered_in_m2    158659 non-null float64
surface_total_in_m2      127116 non-null float64
cosa                     408626 non-null int32
otro                     408626 non-null int32
dtypes: float64(7), int32(2), object(5)
memory usage: 43.6+ MB


In [41]:
train_price = train['price_aprox_usd']

In [42]:
del train['price_aprox_usd']

In [43]:
train = train[['cosa','otro']]

In [44]:
train

,cosa,otro
0,3,582
1,2,589
2,3,195
3,3,195
4,3,249
5,2,613
6,3,162
7,3,195
8,2,225
10,3,667


In [45]:
#train = train[['floor','rooms']]

In [46]:
#train = train.fillna(0)

In [47]:
#train['floor'].value_counts(dropna=False)

In [48]:
#train['rooms'].value_counts(dropna=False)

In [49]:
#train = train['rooms']

In [50]:
x_train, x_test, y_train, y_test = train_test_split(train, train_price, test_size = 0.4, random_state = 0)

knn = KNeighborsRegressor(n_neighbors=10)

knn.fit(x_train, y_train)
pred = knn.predict(x_test)

rms = sqrt(mean_squared_error(pred, y_test))

In [51]:
rms

315691.2704093166

In [52]:
test

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.0,67.0,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,NaN,NaN,47.0,47.0,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...


In [53]:
testKNN = test

In [54]:
testKNN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [55]:
testKNN['cosa'] = testKNN['property_type'].rank(method='dense', ascending=False).astype(int)
testKNN['otro'] = testKNN['place_name'].rank(method='dense', ascending=False).astype(int)

In [56]:
testKNN = testKNN[['cosa','otro']]

In [57]:
x_train, x_test, y_train, y_test = train_test_split(train, train_price, test_size = 0.4, random_state = 0)

knn = KNeighborsRegressor(n_neighbors=10)

knn.fit(x_train, y_train)
pred = knn.predict(testKNN)

#rms2 = sqrt(mean_squared_error(pred, y_test))

In [58]:
pred

array([ 134300.   ,  149000.   ,  191427.744, ...,   85590.   ,
        208500.   ,  201218.9  ])

In [59]:
test['price_usd'] = pred

In [60]:
test

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,cosa,otro,price_usd
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,2,71,134300.000
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...,2,194,149000.000
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...,2,87,191427.744
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2,183,256100.000
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...,2,183,256100.000
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2,183,256100.000
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2,183,256100.000
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2,183,256100.000
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.0,67.0,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...,2,176,251000.000
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,NaN,NaN,47.0,47.0,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...,2,216,200333.709


In [61]:
testASubir = test[['id','price_usd']]

In [62]:
testASubir

,id,price_usd
0,3632,134300.000
1,3633,149000.000
2,2263404,191427.744
3,2263405,256100.000
4,2263406,256100.000
5,2263407,256100.000
6,2263408,256100.000
7,2263409,256100.000
8,2263410,251000.000
9,2263411,200333.709


In [63]:
testASubir.to_csv('data/SubmissionKNN.csv',index=False)

In [65]:
train

,cosa,otro
0,3,582
1,2,589
2,3,195
3,3,195
4,3,249
5,2,613
6,3,162
7,3,195
8,2,225
10,3,667


In [67]:
train_price.count()

408626

# Random forest regressor

In [70]:
#Random Forest Regression
rfr = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 2, random_state = 0)

x1_train, x1_test, y1_train, y1_test = train_test_split(train, train_price, test_size = 0.75, random_state = 24)

rfr.fit(x1_train,y1_train)

prediction_rfr = rfr.predict(x1_test)
rms = sqrt(mean_squared_error(prediction_rfr, y1_test))
print "Random Forest aplicado"
rms

Random Forest aplicado


331980.31138098537

# GBR

In [71]:
#gradient boosting regresion
gbr = GradientBoostingRegressor()
x2_train, x2_test, y2_train, y2_test = train_test_split(train, train_price, test_size = 0.75, random_state = 24)
gbr.fit(x2_train,y2_train)
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
#prediccin y score
prediction_gbr = gbr.predict(x2_test)
rms = sqrt(mean_squared_error(prediction_gbr, y2_test))
rms

336685.39965370053